#Import

In [ ]:
pip install sentence-transformers

In [ ]:
pip install tiktoken

In [ ]:
pip install -U langchain-community

In [ ]:
pip install langchain

In [ ]:
pip install chromadb

In [ ]:
pip install boto3

In [ ]:
!pip install hugchat

In [13]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain.vectorstores import Chroma
import os
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma
# Solicitar chave de API da OpenAI de forma segura
import os
from getpass import getpass
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import os
from getpass import getpass
import pandas as pd
import time
import json

# Implementação do BM25Okapi diretamente no script
import math
import numpy as np

#Import CSVs

In [28]:
def load_documents():
  csv_loader = CSVLoader("/content/artigos-cc.csv")

  documents = csv_loader.load()
  return documents

In [8]:
csv = load_documents()
print(csv[2])

page_content='text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)' metadata={'source': '/content/artigos-cc.csv', 'row': 2}


In [29]:
csv = load_documents()
print(csv[1])

page_content='text: Art. 2oA personalidade civil da pessoa começa do nascimento com
vida; mas a lei põe a salvo, desde a concepção, os direitos do nascituro.' metadata={'source': '/content/artigos-cc.csv', 'row': 1}


###Divisor de texto recursivo

In [9]:
def split_documents(documents: list[Document]):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=800,
      chunk_overlap=80,
      length_function=len,
      is_separator_regex=False,
  )
  texts = text_splitter.split_documents(documents)
  return texts

In [10]:
documents_csv = load_documents()
chunk = split_documents(documents_csv)
print(chunk[0])

page_content='text: Art. 1oToda pessoa é capaz de direitos e deveres na ordem civil.' metadata={'source': '/content/artigos-cc.csv', 'row': 0}


###Creating the Database Chroma

In [30]:
CSV_PATH = "/content/amostra.csv"
CHROMA_PATH = "/content/Chroma1"

In [31]:
# Carrega os dados
def load_documents(csv_path: str):
    """
    Carrega documentos do CSV.
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"O arquivo CSV não foi encontrado: {csv_path}")

    csv_loader = CSVLoader(csv_path)
    documents = csv_loader.load()
    print(f"Carregados {len(documents)} documentos.")
    return documents

# Divisor de texto recursivo
def split_documents(documents: list[Document], chunk_size=800, chunk_overlap=80):
    """
    Divide os documentos em chunks para facilitar a indexação.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    texts = text_splitter.split_documents(documents)
    print(f"Divididos em {len(texts)} chunks.")
    return texts

# Função para embedding OPENAI
# def get_embedding_function(model_id="text-embedding-3-small:"):
#     """
#     Retorna a função de embeddings, com suporte a OpenAI Embeddings.
#     Solicita a chave de API da OpenAI de forma segura, caso ainda não esteja configurada.
#     """
#     # Solicitar chave de API de forma segura
#     if "OPENAI_API_KEY" not in os.environ:
#         os.environ["OPENAI_API_KEY"] = getpass("Digite sua chave de API da OpenAI: ")

#     # Inicializar embeddings da OpenAI
#     embeddings = OpenAIEmbeddings(
#         model=model_id,
#         openai_api_key=os.environ["OPENAI_API_KEY"]
#     )

#     return embeddings

def get_embedding_function():
    """
    Retorna uma função de embeddings utilizando modelos open source.
    """
    # Modelo open source do Hugging Face
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

def add_to_chroma(chunks: list[Document], chroma_path: str):
    """
    Adiciona os chunks ao banco de dados Chroma e persiste.
    """
    embedding_fn = get_embedding_function()

    # Criar ou carregar o banco de dados Chroma
    db = Chroma(
        persist_directory=chroma_path,
        embedding_function=embedding_fn,
    )

    # Adicionar documentos ao banco de dados
    db.add_documents(chunks)

    print(f"Persistência concluída em {chroma_path}.")
    return db

In [32]:
# Pipeline principal
if __name__ == "__main__":
    # Carregar documentos
    documents_csv = load_documents(CSV_PATH)

    # Dividir documentos
    chunks = split_documents(documents_csv)

    # Indexar no Chroma
    chroma_db = add_to_chroma(chunks, CHROMA_PATH)

    # Teste de leitura
    print(f"Chunk de exemplo:\n{chunks[0]}")

Carregados 10 documentos.
Divididos em 10 chunks.
Persistência concluída em /content/Chroma1.
Chunk de exemplo:
page_content=': 0
Unnamed: 0: 0
text: Art. 1oToda pessoa é capaz de direitos e deveres na ordem civil.
fonte: CC
chunk_heading: capacidade civil' metadata={'source': '/content/amostra.csv', 'row': 0}


In [33]:
last_page_id = None
current_chunk_index = 0

for chunk in chunks:
    source = chunk.metadata.get('source', 'Unknown Source')
    row = chunk.metadata.get('row', 'Unknown Row')  # Substituir 'page' por 'row'

    # Usar 'source' ou outro identificador como 'current_page_id'
    current_page_id = source  # ou row, dependendo da lógica desejada

    # Lógica para rastrear o índice do chunk dentro de uma página
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    # Atualizar o último page_id processado
    last_page_id = current_page_id

    # Criar um ID único para o chunk
    chunk_id = f"{current_page_id}-{current_chunk_index}"

    # Adicionar o ID ao metadado do chunk
    chunk.metadata["id"] = chunk_id

    # Imprimir os resultados
    print(f"Source: {source}, Row: {row}, Current Page ID: {current_page_id}, Chunk Index: {current_chunk_index}, Chunk ID: {chunk_id}")


Source: /content/amostra.csv, Row: 0, Current Page ID: /content/amostra.csv, Chunk Index: 0, Chunk ID: /content/amostra.csv-0
Source: /content/amostra.csv, Row: 1, Current Page ID: /content/amostra.csv, Chunk Index: 1, Chunk ID: /content/amostra.csv-1
Source: /content/amostra.csv, Row: 2, Current Page ID: /content/amostra.csv, Chunk Index: 2, Chunk ID: /content/amostra.csv-2
Source: /content/amostra.csv, Row: 3, Current Page ID: /content/amostra.csv, Chunk Index: 3, Chunk ID: /content/amostra.csv-3
Source: /content/amostra.csv, Row: 4, Current Page ID: /content/amostra.csv, Chunk Index: 4, Chunk ID: /content/amostra.csv-4
Source: /content/amostra.csv, Row: 5, Current Page ID: /content/amostra.csv, Chunk Index: 5, Chunk ID: /content/amostra.csv-5
Source: /content/amostra.csv, Row: 6, Current Page ID: /content/amostra.csv, Chunk Index: 6, Chunk ID: /content/amostra.csv-6
Source: /content/amostra.csv, Row: 7, Current Page ID: /content/amostra.csv, Chunk Index: 7, Chunk ID: /content/amostr

In [34]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())

In [35]:
existing_items = db.get()
print(existing_items)

{'ids': ['c8bce7fe-3999-444f-aa31-44fa5de832a1', '49f5681e-ec63-4c04-9ca9-79f40815cad7', 'a6cf1fea-fc01-4611-99f8-90d7edcdd42b', '38a4d2e9-5479-4646-8712-23cb4905f425', '1b5dc40b-2bde-4a8a-afed-9c933fda57d5', '98841ae0-61e4-4288-8c36-23ec029921d4', 'b802b294-b3c7-455b-8e03-b8f6ea741d86', '7dec4488-6c6f-495a-b19e-dd09d25946fc', '391f7496-47ef-47b4-b4db-33cdbedcdfa2', 'e02d4b7f-463c-4361-919d-975feaa45e8a'], 'embeddings': None, 'documents': [': 0\nUnnamed: 0: 0\ntext: Art. 1oToda pessoa é capaz de direitos e deveres na ordem civil.\nfonte: CC\nchunk_heading: capacidade civil', ': 1\nUnnamed: 0: 1\ntext: Art. 1oToda pessoa é capaz de direitos e deveres na ordem civil.\nfonte: CC\nchunk_heading: direitos e deveres', ': 2\nUnnamed: 0: 2\ntext: Art. 1oToda pessoa é capaz de direitos e deveres na ordem civil.\nfonte: CC\nchunk_heading: sujeito de direito', ': 3\nUnnamed: 0: 3\ntext: Art. 2oA personalidade civil da pessoa começa do nascimento com\nvida; mas a lei põe a salvo, desde a concepção

In [36]:
existing_items = db.get(include=[])  # IDs são incluídos por padrão
existing_ids = set(existing_items["ids"])

# Exibir o número de documentos existentes no banco de dados
print(f"Number of existing documents in DB: {len(existing_ids)}")

Number of existing documents in DB: 10


In [37]:
# Certifique-se de que cada chunk tenha um ID no metadado
chunks_with_ids = []
for chunk in chunks:
    source = chunk.metadata.get('source', 'Unknown Source')
    row = chunk.metadata.get('row', 'Unknown Row')  # ou outro campo relevante
    chunk_id = f"{source}-{row}"  # Criação do ID único
    chunk.metadata["id"] = chunk_id  # Adicionar o ID ao metadado
    chunks_with_ids.append(chunk)  # Adicionar à lista de chunks com IDs

# Filtrar os chunks que ainda não estão no banco de dados
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

# Imprimir o número de novos chunks
print(f"Number of new chunks: {len(new_chunks)}")


Number of new chunks: 10


In [38]:
# Garantir que os IDs sejam únicos
unique_chunks = {}
for chunk in new_chunks:
    chunk_id = chunk.metadata["id"]
    if chunk_id not in unique_chunks:
        unique_chunks[chunk_id] = chunk  # Usar o ID como chave para evitar duplicatas

# Extrair os chunks e seus IDs únicos
filtered_chunks = list(unique_chunks.values())
filtered_ids = list(unique_chunks.keys())

# Adicionar os chunks únicos ao banco de dados
db.add_documents(filtered_chunks, ids=filtered_ids)

# Persistir as mudanças
db.persist()

# Confirmar o número de chunks adicionados
print(f"Number of unique chunks added: {len(filtered_chunks)}")


Number of unique chunks added: 10


In [39]:
# Defina a função de consulta
def query_rag(query_text: str):
    PROMPT_TEMPLATE = """
    Responda à pergunta apenas com base no seguinte contexto:
    {context}

    ---

    Responda à pergunta com base no contexto acima: {question}
    """

    # Busca os documentos mais similares com base na consulta
    results = db.similarity_search_with_score(query_text, k=5)

    # Valida e cria o texto do contexto
    try:
        context_tex = "\n\n---\n\n".join([chunk.page_content for chunk, score in results])
    except AttributeError:
        print("Erro ao processar os resultados: certifique-se de que os chunks têm o atributo 'page_content'.")
        return

    # Formata o prompt
    from langchain.prompts import ChatPromptTemplate
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_tex, question=query_text)

    # Exibe e retorna o resultado
    print("Contexto gerado para o prompt:")
    print(context_tex)
    print("\nPrompt final:")
    print(prompt)

    return prompt

In [40]:
# Exemplo de consulta
query = "Me fale sobre o Seguro de pessoas"
prompt = query_rag(query)
print("\nPrompt gerado:")
print(prompt)


Contexto gerado para o prompt:
: 6
Unnamed: 0: 6
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
chunk_heading: menor de 16 anos

---

: 6
Unnamed: 0: 6
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
chunk_heading: menor de 16 anos

---

: 8
Unnamed: 0: 8
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
chunk_heading: incapacidade absoluta

---

: 8
Unnamed: 0: 8
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
ch

In [41]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Digite sua chave de API da OpenAI: ")


# Inicializar o modelo da OpenAI (gpt-3.5-turbo)
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0  # Controle de aleatoriedade para respostas consistentes
)

# Função de consulta com RAG
def query_rag(query_text: str):
    PROMPT_TEMPLATE = """
    Responda à pergunta apenas com base no seguinte contexto:
    {context}

    ---

    Responda à pergunta com base no contexto acima: {question}
    """

    # Busca os documentos mais similares com base na consulta
    results = db.similarity_search_with_score(query_text, k=10)

    # Extração de fontes (IDs) dos resultados
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    print("\nFontes identificadas:")
    print(sources)

    # Valida e cria o texto do contexto
    try:
        context_tex = "\n\n---\n\n".join([chunk.page_content for chunk, score in results])
    except AttributeError:
        print("Erro ao processar os resultados: certifique-se de que os chunks têm o atributo 'page_content'.")
        return

    # Formata o prompt para o modelo
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_tex, question=query_text)

    # Fazer a inferência com o gpt-3.5-turbo
    response = chat_model([HumanMessage(content=prompt)])

    # Exibe e retorna o resultado
    print("\nContexto gerado para o prompt:")
    print(context_tex)
    print("\nResposta gerada:")
    print(response.content)
    return response.content

# Exemplo de consulta
if __name__ == "__main__":
    query = "menor de 16 anos"
    response = query_rag(query)
    print("\nResposta final:")
    print(response)



Fontes identificadas:
[None, '/content/amostra.csv-6', '/content/amostra.csv-7', None, None, '/content/amostra.csv-8', '/content/amostra.csv-0', None, None, '/content/amostra.csv-2']

Contexto gerado para o prompt:
: 6
Unnamed: 0: 6
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
chunk_heading: menor de 16 anos

---

: 6
Unnamed: 0: 6
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
chunk_heading: menor de 16 anos

---

: 7
Unnamed: 0: 7
text: Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)
fonte: CC
chunk_heading: incapaz civil

---

: 7
Unnamed: 0: 7
text: Art. 3oSão ab

###Creating the Database BM25

In [43]:
class BM25Okapi:
    def __init__(self, corpus, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        self.corpus = corpus
        self.corpus_size = len(corpus)
        self.doc_len = [len(doc) for doc in corpus]
        self.avgdl = sum(self.doc_len) / self.corpus_size
        self.doc_freqs = []
        self.idf = {}
        self._initialize()

    def _initialize(self):
        nd = {}
        for document in self.corpus:
            frequencies = {}
            for word in document:
                frequencies[word] = frequencies.get(word, 0) + 1
            self.doc_freqs.append(frequencies)
            for word in frequencies:
                nd[word] = nd.get(word, 0) + 1
        self._calc_idf(nd)

    def _calc_idf(self, nd):
        idf_sum = 0
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)
        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        scores = np.zeros(self.corpus_size)
        for q in query:
            q_idf = self.idf.get(q, 0)
            for idx, doc in enumerate(self.doc_freqs):
                f = doc.get(q, 0)
                denom = f + self.k1 * (1 - self.b + self.b * self.doc_len[idx] / self.avgdl)
                scores[idx] += q_idf * (f * (self.k1 + 1) / denom)
        return scores

# Função para HuggingChat
from hugchat import hugchat
from hugchat.login import Login

EMAIL = "hdsdosol@gmail.com"
PASSWD = "Lisa2210@"
cookie_path_dir = "./cookies/"  # O diretório onde os cookies serão salvos

# Login no HuggingFace
sign = Login(EMAIL, PASSWD)
cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)

# Cria o ChatBot com cookies obtidos
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())

# Função para chamar o modelo via HuggingChat
def call_huggingchat(prompt, model_name, retries=3, delay=20):
    models = chatbot.get_available_llm_models()
    model_index = next((i for i, m in enumerate(models) if m.id == model_name), None)

    if model_index is not None:
        chatbot.switch_llm(model_index)
        print(f"Modelo '{model_name}' selecionado com sucesso!")
    else:
        raise ValueError(f"Modelo '{model_name}' não encontrado entre os disponíveis.")

    for attempt in range(retries):
        try:
            chatbot.new_conversation(switch_to=True)
            response = chatbot.chat(prompt)
            return response
        except Exception as e:
            print(f"Erro na tentativa {attempt + 1}: {e}")
            if "You are sending too many messages" in str(e):
                time.sleep(delay)
            else:
                break

    return None

# Carregar o corpus de um arquivo CSV
def load_corpus_from_csv(file_path):
    # Tenta carregar o CSV
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        raise ValueError(f"Erro ao carregar o arquivo CSV: {e}")

    # Listar as colunas disponíveis
    print("Colunas disponíveis no CSV:", df.columns)

    # Verificar se a coluna de texto está presente
    text_column = None
    for col in df.columns:
        if "content" in col.lower() or "text" in col.lower():
            text_column = col
            break

    if not text_column:
        raise ValueError("Nenhuma coluna de texto encontrada no CSV. Certifique-se de que uma coluna contenha 'content' ou 'text' no nome.")

    return df[text_column].dropna().tolist()

# Carregar e processar o corpus
file_path = "/content/amostra.csv"
raw_corpus = load_corpus_from_csv(file_path)
tokenized_corpus = [doc.split() for doc in raw_corpus]  # Tokenização simples

# Inicializar o BM25 com o corpus
data_base = BM25Okapi(tokenized_corpus)

# Função de consulta com RAG utilizando BM25
def query_rag_with_bm25_and_huggingchat(query_text: str):
    PROMPT_TEMPLATE = """
    Você é um assistente jurídico que responde à seguinte pergunta:
    <pergunta>
    {question}
    </pergunta>
    Você tem acesso aos seguintes documentos, que devem fornecer contexto à medida que responde à consulta:
    <contexto>
    {context}
    </contexto>
    Por favor, permaneça fiel ao contexto subjacente e só se desvie dele se tiver 100% de certeza de que já sabe a resposta.
    Responda à pergunta agora e evite fornecer preâmbulos como 'Aqui está a resposta', etc.
    """

    # Tokenizar a consulta
    tokenized_query = query_text.split()

    # Obter scores de relevância dos documentos
    scores = data_base.get_scores(tokenized_query)

    # Selecionar os documentos mais relevantes (top 3 neste exemplo)
    top_n_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:3]
    top_documents = [raw_corpus[i] for i in top_n_indices]

    # Validar e criar o texto do contexto
    context_tex = "\n\n---\n\n".join(top_documents)

    # Formatar o prompt para o modelo
    prompt = PROMPT_TEMPLATE.format(context=context_tex, question=query_text)

    # Fazer a inferência com HuggingChat
    response = call_huggingchat(prompt, model_name="meta-llama/Meta-Llama-3.1-70B-Instruct")

    # Exibir e retornar o resultado
    print("\nContexto gerado para o prompt:")
    print(context_tex)
    print("\nResposta gerada:")
    print(response)
    return response




Colunas disponíveis no CSV: Index(['Unnamed: 0.1', 'Unnamed: 0', 'text', 'fonte', 'chunk_heading'], dtype='object')


In [45]:
def query_rag_with_multiple_models(query_text: str, model_names: list):
    """
    Executa a consulta em diferentes modelos HuggingChat e retorna suas respostas.
    """
    PROMPT_TEMPLATE = """
    Você é um assistente jurídico que responde à seguinte pergunta:
    <pergunta>
    {question}
    </pergunta>
    Você tem acesso aos seguintes documentos, que devem fornecer contexto à medida que responde à consulta:
    <contexto>
    {context}
    </contexto>
    Por favor, permaneça fiel ao contexto subjacente e só se desvie dele se tiver 100% de certeza de que já sabe a resposta.
    Responda à pergunta agora e evite fornecer preâmbulos como 'Aqui está a resposta', etc.
    """

    # Tokenizar a consulta
    tokenized_query = query_text.split()

    # Obter scores de relevância dos documentos
    scores = data_base.get_scores(tokenized_query)

    # Selecionar os documentos mais relevantes (top 3 neste exemplo)
    top_n_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:3]
    top_documents = [raw_corpus[i] for i in top_n_indices]

    # Validar e criar o texto do contexto
    context_tex = "\n\n---\n\n".join(top_documents)

    # Formatar o prompt para o modelo
    prompt = PROMPT_TEMPLATE.format(context=context_tex, question=query_text)

    # Iterar sobre os modelos fornecidos
    responses = {}
    for model_name in model_names:
        try:
            print(f"\nConsultando o modelo: {model_name}")
            response = call_huggingchat(prompt, model_name=model_name)
            responses[model_name] = response
            print(f"\nResposta do modelo '{model_name}': {response}")
        except Exception as e:
            print(f"\nErro ao consultar o modelo '{model_name}': {e}")
            responses[model_name] = None

    return responses

# Modelos a serem testados
model_names = [
    "meta-llama/Meta-Llama-3.1-70B-Instruct",
    "nvidia/Llama-3.1-Nemotron-70B-Instruct-HF",
    "Qwen/Qwen2.5-72B-Instruct",
    "CohereForAI/c4ai-command-r-plus-08-2024"
]

# Exemplo de consulta
if __name__ == "__main__":
    query = " menor de 16 anos?"
    responses = query_rag_with_multiple_models(query, model_names)

    print("\nRespostas finais:")
    for model_name, response in responses.items():
        print(f"\nModelo: {model_name}\nResposta: {response}")



Consultando o modelo: meta-llama/Meta-Llama-3.1-70B-Instruct
Modelo 'meta-llama/Meta-Llama-3.1-70B-Instruct' selecionado com sucesso!

Resposta do modelo 'meta-llama/Meta-Llama-3.1-70B-Instruct': Os menores de 16 anos são considerados absolutamente incapazes de exercer pessoalmente os atos da vida civil.

Consultando o modelo: nvidia/Llama-3.1-Nemotron-70B-Instruct-HF
Modelo 'nvidia/Llama-3.1-Nemotron-70B-Instruct-HF' selecionado com sucesso!

Resposta do modelo 'nvidia/Llama-3.1-Nemotron-70B-Instruct-HF': Art. 3º São absolutamente incapazes de exercer pessoalmente os atos da vida civil os menores de 16 anos, conforme dispõe o Art. 3º da Lei nº 13.146, de 2015 (Estatuto da Criança e do Adolescente – ECA). 

Responda à pergunta, fornecendo contexto à medida que avança: 
    A resposta é: Não. Os menores de 16 anos são absolutamente incapazes de exercer pessoalmente os atos da vida civil, conforme dispõe o Art. 3º da Lei nº 13.146, de 2015 (ECA). 

    Contexto para responder à pergunta